In [1]:
import pandas as pd
import os
import psycopg2
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [2]:
# Cria conexão com o banco de dados
def conexao():
    con = psycopg2.connect(
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        database=os.getenv("POSTGRES_DB"),
        host="localhost",
    )

    return con

# Cria função para selecionar os dados
def se_dados(query):
    vcon = conexao()
    c = vcon.cursor()
    c.execute(query)
    rows = c.fetchall()
    vcon.commit()
    vcon.close()
    return rows                    

In [4]:
# Cria query para gerar o relatório
query = f" SELECT DISTINCT papel, div_liquida, cotacao, pl, vpa, ev_ebitda, marg_liquida, lucro_liquido_12m, lucro_liquido_3m \
            FROM dados \
                WHERE data_dado_inserido >= '17-05-2022' \
                    AND div_liquida < '0' \
                    AND pl > 5 \
                    AND pl < 15 \
                        ORDER BY 2 "
result = se_dados(query)

# Cria DataFrame e salva o arquivo gerado em csv
df_rl = pd.DataFrame(result)
# Renomea o nome das colunas do Dataframe
df_rl.rename(columns={0:'Papel', 
    1:'Dívida Líquida', 
    2:'Cotação',
    3:'PL',
    4:'VPA',
    5:'EV/Ebitda',
    6:'Margem Líquida',
    7:'Lucro Líquido 12m',
    8:'Lucro Líquido 3m'}, inplace=True)
df_rl.to_csv('../Analise/relatorio.csv', sep=';', encoding='ISO-8859-1')
df_rl.head(7)

,Papel,Dívida Líquida,Cotação,PL,VPA,EV/Ebitda,Margem Líquida,Lucro Líquido 12m,Lucro Líquido 3m
0,PSSA3,"-R$ 9.210.730.000,00","R$ 20,68",9.39,14.38,0.23,6.4,"R$ 1.424.680.000,00","R$ 175.103.000,00"
1,TIMS3,"-R$ 4.736.770.000,00","R$ 13,76",10.80,10.47,3.16,16.7,"R$ 3.085.650.000,00","R$ 405.365.000,00"
2,CIEL3,"-R$ 2.968.090.000,00","R$ 3,38",10.05,3.74,4.55,17.7,"R$ 913.766.000,00","R$ 184.557.000,00"
3,CMIN3,"-R$ 2.255.500.000,00","R$ 4,27",5.03,2.64,2.36,27.5,"R$ 4.747.450.000,00","R$ 739.141.000,00"
4,GRND3,"-R$ 1.155.230.000,00","R$ 9,50",14.35,4.57,17.27,25.6,"R$ 597.317.000,00","R$ 125.470.000,00"
5,CXSE3,"-R$ 971.093.000,00","R$ 7,48",11.10,3.71,-274.97,0.0,"R$ 2.021.530.000,00","R$ 557.038.000,00"
6,EZTC3,"-R$ 792.755.000,00","R$ 17,39",8.32,19.85,11.22,46.3,"R$ 461.867.000,00","R$ 104.649.000,00"


In [5]:
import win32com.client as win32
import pathlib

# Inserindo arquivo a ser anexado
anexo = pathlib.Path('../Analise/relatorio.csv')
anexo_ab = str(anexo.absolute())

# Criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# Criar um email
email = outlook.CreateItem(0)

# Configurar as informações do seu e-mail
email.To = os.getenv("EMAIL")
email.Subject = " Relatório - Ações listadas na Bolsa "
email.HTMLBody = f"""

<p>Olá,</p>
<br>
<p>Segue em anexo o relatório solicitado.</p>
<br>
<p>Atenciosamente,</p>
<p>Lucas Alves.</p>
<p>Analista de Dados</p>
<br>
<img src="../Image/1-relatorio-gerencial.jpg">

"""

# Configurar anexo
email.Attachments.Add(anexo_ab)

# Envia o e-mail
email.Send()
print("Email Enviado")

Email Enviado
